In [46]:
import pandas as pd     
import numpy as np 
import glob as glob

Base=[]
csv_files = glob.glob('Data/*.csv')

for filename in csv_files:
    if filename == "Data\\EAM_2020.csv":
        print("ok")
        data = pd.read_csv(filename,sep=",")   
    else:
        data = pd.read_csv(filename,sep=";")
      
    Base.append(data)

c:\Users\User\Documents\TrabajoGrado\env\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (299) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\User\Documents\TrabajoGrado\env\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (182,183,184,185,186,187,188,189,190,191,302) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\User\Documents\TrabajoGrado\env\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,2

ok


In [47]:
EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20 = Base
#Assign each dataframe a variable of the "Base"
Data_Merge = [EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20]
#Create a list of the dataframe

EAM08_G,EAM09_G,EAM10_G,EAM11_G,EAM12_G,EAM13_G,EAM14_G, \
            EAM15_G,EAM16_G,EAM17_G,EAM18_G,EAM19_G,EAM20_G=\
                0,0,0,0,0,0,0,0,0,0,0,0,0

Data_Gruop = [EAM08_G,EAM09_G,EAM10_G,EAM11_G,EAM12_G,EAM13_G,EAM14_G, \
            EAM15_G,EAM16_G,EAM17_G,EAM18_G,EAM19_G,EAM20_G]

In [48]:
def change_name(Data_Merge, Number, change):
    for df in Data_Merge:
        if len(df.columns) > Number: #Verificar si el dataframe tiene al menos 4 columnas
            df.columns.values[Number] = change #Me cambia el name de el label 

    return Data_Merge

In [49]:
#Función para saber si una columna esta presenta en un dataframe
def Exis(Data_Merge,item): 
    columna_presente = True
    for df in Data_Merge:
        if item not in df.columns:
            columna_resente = False
    return columna_presente

In [50]:
#Función para saber si existen datos faltantes en la columna
def exis_colum(Data_Merge,colum):
    missing_col=0
    for i in Data_Merge:
        missing_col += i[colum].isnull().sum()
    return missing_col

In [51]:
#Función para ordenar, por una columna
def Sort(Column,Data_Merge):
    for i, d in enumerate(Data_Merge):
        sorted_df = d.sort_values(by=Column)
        Data_Merge[i] = sorted_df 
    return Data_Merge

In [52]:
def Sum_Var(Column, SUM, UBAC, Data_Gruop, Data_Merge):
    for i, df in enumerate(Data_Gruop):
        total = 0
        for index in df.index:
            total = Data_Merge[i][Data_Merge[i][Column] == index][SUM].sum()
            df.loc[index, UBAC] = total
    return Data_Gruop

In [53]:
Data_Merge = change_name(Data_Merge,3,"CIIU")

In [54]:
Exis(Data_Merge,"CIIU")
Exis(Data_Merge,"VALORVEN") #Identification for VALORVEN if there is in each dataframe of the model

True

In [55]:
exis_colum(Data_Merge,"VALORVEN")
exis_colum(Data_Merge,"CIIU")

0

In [56]:
Data_Merge=Sort("CIIU",Data_Merge)

In [57]:
#Para crear un data frame agrupado de una base de datos
for i in range (len(Data_Merge)):
    index=Data_Merge[i]["CIIU"].drop_duplicates().sort_values()
    Data_Gruop[i]= pd.DataFrame(index=index,columns=["SUM-VALORVEN","SUM-WAGE"])


In [58]:
Data_Gruop=Sum_Var("CIIU","VALORVEN","SUM-VALORVEN",Data_Gruop,Data_Merge,)


In [59]:
Data_Gruop[0]

,SUM-VALORVEN,SUM-WAGE
CIIU,,
1511,4416347167,NaN
1512,604183259,NaN
1521,272569823,NaN
1522,3408849362,NaN
1530,4618616769,NaN
...,...,...
3619,293300838,NaN
3691,8057819,NaN
3693,11414398,NaN
